<h1 align="center">Segmentation: Thresholding and Edge Detection</h1>

In this notebook our goal is to estimate the radius of spherical markers from an image (Cone-Beam CT volume).

We will use two approaches:
1. Segment the fiducial using a thresholding approach, derive the sphere's radius from the segmentation. This approach is solely based on SimpleITK.
2. Localize the fiducial's edges using the Canny edge detector and then fit a sphere to these edges using a least squares approach. This approach is a combination of SimpleITK and R.

It should be noted that all of the operations, filtering and computations, are natively in 3D. This is the "magic" of ITK and SimpleITK at work.

In [ ]:
library(SimpleITK)

source("downloaddata.R")

Load the volume and look at the image (visualization requires window-leveling).

In [ ]:
spherical_fiducials_image <- ReadImage(fetch_data("spherical_fiducials.mha"))
Show(spherical_fiducials_image, "spheres")

After looking at the image you should have identified two spheres. Now select a Region Of Interest (ROI) around the sphere which you want to analyze. 

In [ ]:
roi1 = list(c(280,320), c(65,90), c(8, 30))
roi2 = list(c(200,240), c(65,100), c(15, 40))
mask_value = 255

# Select the roi
roi = roi1
# Update the R roi, SimpleITK indexes are zero based, R indexes start at one  
r_roi = lapply(roi, function(x) x+1)
    
# Create the mask image from an R array    
amask <- array(0, spherical_fiducials_image$GetSize())
xs <- r_roi[[1]][1]:r_roi[[1]][2]
ys <- r_roi[[2]][1]:r_roi[[2]][2]
zs <- r_roi[[3]][1]:r_roi[[3]][2]
amask[xs, ys, zs] <- mask_value

mask <- Cast(as.image(amask), "sitkUInt8")
mask$CopyInformation(spherical_fiducials_image)
Show(LabelOverlay(Cast(IntensityWindowing(spherical_fiducials_image, windowMinimum=-32767, 
                                          windowMaximum=-29611), 
                       "sitkUInt8"), 
                   mask, opacity=0.5))    

## Thresholding based approach

Our region of interest is expected to have a bimodal intensity distribution with high intensities belonging to the spherical marker and low ones to the background. We can thus use Otsu's method for threshold selection to segment the sphere and estimate its radius. 

In [ ]:
# Set pixels that are in [min_intensity,otsu_threshold] to inside_value, values above otsu_threshold are
# set to outside_value. The sphere's have higher intensity values than the background, so they are outside.

inside_value <- 0
outside_value <- 255
number_of_histogram_bins <- 100
mask_output <- TRUE

labeled_result <- OtsuThreshold(spherical_fiducials_image, mask, inside_value, outside_value, 
                                number_of_histogram_bins, mask_output, mask_value)

# Estimate the sphere radius from the segmented image using the LabelShapeStatisticsImageFilter.
label_shape_analysis <- LabelShapeStatisticsImageFilter()
label_shape_analysis$SetBackgroundValue(inside_value)
dummy <- label_shape_analysis$Execute(labeled_result)
cat("The sphere's radius is: ",label_shape_analysis$GetEquivalentSphericalRadius(outside_value),"mm") 

## Edge detection based approach

In this approach we will localize the sphere's edges in 3D using SimpleITK. We then compute the least squares sphere that optimally fits the 3D points using R. The mathematical formulation for this solution is described in this [Insight Journal paper](http://www.insight-journal.org/download/viewpdf/769/1/download). We also look at a weighted version of least squares fitting using R's linear model fitting approach. 


In [ ]:
# Create a cropped version of the original image.
sub_image = spherical_fiducials_image[r_roi[[1]][1]:r_roi[[1]][2],
                                      r_roi[[2]][1]:r_roi[[2]][2],
                                      r_roi[[3]][1]:r_roi[[3]][2]]

# Edge detection on the sub_image with appropriate thresholds and smoothing.
edges <- CannyEdgeDetection(Cast(sub_image, "sitkFloat32"), 
                            lowerThreshold=0.0, 
                            upperThreshold=200.0, 
                            variance = c(5.0, 5.0, 5.0))

# Get the 3D location of the edge points
edge_indexes <- which(as.array(edges)==1.0, arr.ind=TRUE)
# Always remember to modify indexes when shifting between native R operations and SimpleITK operations
physical_points <- t(apply(edge_indexes - 1, MARGIN=1,
                           sub_image$TransformIndexToPhysicalPoint))

Visually inspect the results of edge detection, just to make sure. Note that because SimpleITK is working in the
physical world (not pixels, but mm) we can easily transfer the edges localized in the cropped image to the original.

In [ ]:
edge_label <- Image(spherical_fiducials_image$GetSize(), "sitkUInt8")
edge_label$CopyInformation(spherical_fiducials_image)
e_label <- 255
apply(physical_points, 
      MARGIN=1, 
      function(x, img, label) img$SetPixel(img$TransformPhysicalPointToIndex(x),label), 
      img=edge_label, 
      label=e_label)
    
Show(LabelOverlay(Cast(IntensityWindowing(spherical_fiducials_image, windowMinimum=-32767, windowMaximum=-29611),
                       "sitkUInt8"), 
                  edge_label, opacity=0.5))

Setup and solve linear equation system.

In [ ]:
A <- -2 * physical_points
A <- cbind(A, 1)
b <- -rowSums(physical_points^2)
x <- solve(qr(A, LAPACK=TRUE), b)
cat("The sphere's center is: ", x, "\n")
cat("The sphere's radius is: ", sqrt(x[1:3] %*% x[1:3] - x[4]), "\n")

Now, solve using R's linear model fitting. We also weigh the edge points based on the gradient magnitude.

In [ ]:
gradient_magnitude = GradientMagnitude(sub_image)
grad_weights = apply(edge_indexes-1, MARGIN=1, gradient_magnitude$GetPixel)

df <- data.frame(Y=rowSums(physical_points^2), x=physical_points[, 1],
                 y=physical_points[, 2], z=physical_points[, 3])
fit <- lm(Y ~ x + y + z, data=df, weights=grad_weights)

center <- coefficients(fit)[c("x", "y", "z")] / 2
radius <- sqrt(coefficients(fit)["(Intercept)"] + sum(center^2))
cat("The sphere's center is: ", center, "\n")
cat("The sphere's radius is: ", radius, "\n")

## You've made it to the end of the notebook, so what is the sphere's radius?

The radius is 3mm. 